In [1]:
import pandas as pd
from pyspark.sql import SparkSession
import json
import base64
import os
import pandas as pd
from tqdm import tqdm # Opcional: para ver una barra de progreso

In [ ]:
import ijson
import base64
import os
import csv

# --- CONFIGURACIÓN ---
input_json = r"Base de datos\\AQ_database_filtered_june_2087.json"
folder_fotos = "imagenes_sensores"
output_csv = "dataset_aire_limpio.csv"

if not os.path.exists(folder_fotos):
    os.makedirs(folder_fotos)

print("🚀 Iniciando extracción por streaming (Modo Ahorro de RAM)...")

# Definimos las columnas para el CSV
columnas = [
    "_id", "Fecha de inicio", "Fecha de recogida", 
    "Localización longitud", "Localización latitud", 
    "Número de contornos detectados", "Porcentaje de área detectada", 
    "Concentración estándar", "Nivel de polución", "Imagen de entrada"
]

# Abrimos el CSV para ir escribiendo fila a fila
with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=columnas)
    writer.writeheader()

    # Abrimos el JSON gigante en modo lectura binaria para ijson
    with open(input_json, 'rb') as f:
        # 'item' asume que el JSON empieza con [ {resgistro}, {registro} ... ]
        # Si el JSON tiene otra raíz, cámbialo (ej. 'results.item')
        registros = ijson.items(f, 'item')
        
        for i, registro in enumerate(registros):
            # 1. Obtener ID
            id_raw = registro.get('_id', f'img_{i}')
            id_str = id_raw['$oid'] if isinstance(id_raw, dict) else str(id_raw)
            
            # 2. Procesar imagen
            img_b64 = registro.get('Imagen de entrada', '')
            nombre_archivo = f"{id_str}.jpg"
            
            if img_b64:
                try:
                    if "," in img_b64:
                        img_b64 = img_b64.split(",")[1]
                    
                    ruta_img = os.path.join(folder_fotos, nombre_archivo)
                    with open(ruta_img, "wb") as img_file:
                        img_file.write(base64.b64decode(img_b64))
                    
                    foto_ref = nombre_archivo
                except:
                    foto_ref = "ERROR_CONVERSION"
            else:
                foto_ref = "SIN_IMAGEN"

            # 3. Limpiar el registro y escribir en el CSV inmediatamente
            # Solo guardamos las columnas que nos interesan
            fila_csv = {col: registro.get(col, '') for col in columnas}
            fila_csv['Imagen de entrada'] = foto_ref
            
            # Si el _id era un objeto raro de Mongo, lo aplanamos a texto
            fila_csv['_id'] = id_str 

            writer.writerow(fila_csv)
            
            if (i + 1) % 100 == 0:
                print(f"✅ Procesados {i + 1} registros...")

print(f"\n✨ ¡Completado con éxito! Ya puedes borrar el JSON de 6GB si quieres.")

🚀 Iniciando extracción por streaming (Modo Ahorro de RAM)...


FileNotFoundError: [Errno 2] No such file or directory: 'Base de datos\\AQ_database_filtered_june_2087.json'